In [1]:
# data parameters | 数据文件路径
#parser.add_argument('--test_csv_path', type=str)
test_path="/rds/user/hpcpin1/hpc-work/SegProject/Datasets/Cytomine/Cephalo/Train"

# model load dir path | 存放模型的文件夹路径
#parser.add_argument('--load_weight_path', type=str)
load_weight_path="/rds/user/hpcpin1/hpc-work/CL-Detection2023/model_noaug_nonorm__L1_18091358/checkpoint_epoch_390.pt"
# model hyper-parameters: image_width and image_height
#parser.add_argument('--image_width', type=int, default=512)
#parser.add_argument('--image_height', type=int, default=512)
image_width=512

# model test hyper-parameters
#parser.add_argument('--cuda_id', type=int, default=0)
cuda_id=0
#parser.add_argument('--model_name', type=str, default='UNet')
model_name="UNet"
# result & save

#parser.add_argument('--save_image', type=bool, default=True)
save_image=False
#parser.add_argument('--save_image_dir', type=str, default='./visualize/')
save_image_dir='./visualize/'

In [2]:
# !/usr/bin/env python
# -*- coding:utf-8 -*-
# author: zhanghongyuan2017@email.szu.edu.cn

import os
import tqdm
import torch
import argparse
import numpy as np
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

import warnings
warnings.filterwarnings('ignore')

#from utils.tranforms import Rescale, RandomHorizontalFlip, ToTensor
#from utils.dataset import CephXrayDataset
from utils.model import load_model
from utils.losses import load_loss


from utils.cldetection_utils import check_and_make_dir

import sys
print("current directory :", os.getcwd())
sys.path.insert(1, '/rds/user/hpcpin1/hpc-work/SegProject')
print(sys.path)
from utils.dataLoader import LandmarksDataset_NoNorm, ToTensorCL, ToTensorLH, Rescale, RandomScale, AugColor, Rotate

##########VALIDATION CHECK
train_csv_path="/rds/user/hpcpin1/hpc-work/SegProject/Datasets/Cytomine/Cephalo/Train"
valid_csv_path="/rds/user/hpcpin1/hpc-work/SegProject/Datasets/Cytomine/Cephalo/Val"



img_path = os.path.join(train_csv_path, 'Images')
label_path = os.path.join(train_csv_path, 'landmarks')


train_dataset = LandmarksDataset_NoNorm(img_path=img_path,
                                 label_path=label_path,
                                transform = transforms.Compose([
                                             Rescale(512),
                                             ToTensorCL()])
                                 )

img_path = os.path.join(valid_csv_path, 'Images')
label_path = os.path.join(valid_csv_path, 'landmarks')
val_dataset = LandmarksDataset_NoNorm(img_path=img_path,
                                 label_path=label_path,
                                 transform = transforms.Compose([
                                             Rescale(512),
                                             ToTensorCL()])
                                 )



train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 8, shuffle = True, num_workers = 2)
valid_loader = torch.utils.data.DataLoader(val_dataset, batch_size = 8, num_workers = 2)

# train_loader = DataLoader(train_dataset,
#                           batch_size=config.batch_size,
#                           shuffle=True,
#                           num_workers=config.num_workers)
# valid_loader = DataLoader(valid_dataset,
#                           batch_size=config.batch_size_valid,
#                           shuffle=False,
#                           num_workers=config.num_workers)

# load model
# GPU device
gpu_id = cuda_id
os.environ["CUDA_VISIBLE_DEVICES"] = "{}".format(gpu_id)
device = torch.device('cuda:{}'.format(gpu_id) if torch.cuda.is_available() else 'cpu')

# load model
model = load_model(model_name='UNet')
model.load_state_dict(torch.load(load_weight_path, map_location=device))
model = model.to(device)

# optimizer and StepLR scheduler
# optimizer = torch.optim.Adam(model.parameters(),
#                              lr=config.lr,
#                              betas=(config.beta1, config.beta2))
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
#                                             step_size=config.scheduler_step_size,
#                                             gamma=config.scheduler_gamma)

# model loss function
loss_fn = load_loss(loss_name='L1')

# model training preparation
train_losses = []
valid_losses = []
best_loss = 1e10
num_epoch_no_improvement = 0


# start to train and valid


current directory : /rds/user/hpcpin1/hpc-work/CL-Detection2023
['/rds/user/hpcpin1/hpc-work/CL-Detection2023', '/rds/user/hpcpin1/hpc-work/SegProject', '/home/hpcpin1/.conda/envs/torch2/lib/python310.zip', '/home/hpcpin1/.conda/envs/torch2/lib/python3.10', '/home/hpcpin1/.conda/envs/torch2/lib/python3.10/lib-dynload', '', '/home/hpcpin1/.conda/envs/torch2/lib/python3.10/site-packages']


In [3]:
image, heatmap, landmarks = next(iter(train_loader))   


In [4]:
image.shape

torch.Size([8, 1, 512, 512])

In [5]:
image2, heatmap2 = image.float().to(device), heatmap.float().to(device)

In [6]:
image.shape

torch.Size([8, 1, 512, 512])

In [7]:
with torch.no_grad():
    output = model(image2)

In [8]:
output.shape

torch.Size([8, 19, 512, 512])

In [9]:
loss = loss_fn(output, heatmap2)
print(loss)

tensor(0.0770, device='cuda:0')


In [10]:
loss = loss_fn(output, output)
print(loss)

tensor(0., device='cuda:0')


In [11]:
loss = loss_fn(heatmap2, heatmap2)
print(loss)

tensor(0., device='cuda:0')


In [12]:
loss = loss_fn(heatmap, heatmap)
print(loss)

tensor(0., dtype=torch.float64)


In [13]:
image3, heatmap3, landmarks3 = next(iter(train_loader))  
image3, heatmap3 = image3.float().to(device), heatmap3.float().to(device)
with torch.no_grad():
    output3 = model(image3)
print(loss_fn(heatmap3, heatmap3))
print(loss_fn(output3, output3))
print(loss_fn(heatmap3, output3))
print(loss_fn(heatmap2, heatmap3))


tensor(0., device='cuda:0')
tensor(0., device='cuda:0')
tensor(0.0770, device='cuda:0')
tensor(0.0024, device='cuda:0')


In [14]:
image4, heatmap4, landmarks4 = next(iter(train_loader))  
image4, heatmap4 = image4.float().to(device), heatmap4.float().to(device)
with torch.no_grad():
    output4 = model(image4)
print("h4/h4", loss_fn(heatmap4, heatmap4))
print("o4/o4",loss_fn(output4, output4))
print("h4/o4",loss_fn(heatmap4, output4))
print("h4/h3",loss_fn(heatmap4, heatmap3))

h4/h4 tensor(0., device='cuda:0')
o4/o4 tensor(0., device='cuda:0')
h4/o4 tensor(0.0770, device='cuda:0')
h4/h3 tensor(0.0025, device='cuda:0')


In [15]:
image4, heatmap4, landmarks4 = next(iter(train_loader))  
image4, heatmap4 = image4.float().to(device), heatmap4.float().to(device)
with torch.no_grad():
    output4 = model(image4)
print("h4/o4",loss_fn(heatmap4, output4))
print("h4/h4", loss_fn(heatmap4, heatmap4))
print("o4/o4",loss_fn(output4, output4))
print("h4/h3",loss_fn(heatmap4, heatmap3))

h4/o4 tensor(0.0770, device='cuda:0')
h4/h4 tensor(0., device='cuda:0')
o4/o4 tensor(0., device='cuda:0')
h4/h3 tensor(0.0023, device='cuda:0')


In [16]:
image4, heatmap4, landmarks4 = next(iter(train_loader))  
image4, heatmap4 = image4.float().to(device), heatmap4.float().to(device)
with torch.no_grad():
    output4 = model(image4)
print("h4/o4",loss_fn(heatmap4, output4))
print("o4/h4",loss_fn(output4, heatmap4))


h4/o4 tensor(0.0770, device='cuda:0')
o4/h4 tensor(0.0770, device='cuda:0')


In [17]:
image4, heatmap4, landmarks4 = next(iter(train_loader))  
image4, heatmap4 = image4.float().to(device), heatmap4.float().to(device)
with torch.no_grad():
    output4 = model(image4)
print(loss_fn(output4, heatmap4))


tensor(0.0770, device='cuda:0')


In [18]:
image4, heatmap4, landmarks4 = next(iter(train_loader))  
image4, heatmap4 = image4.float().to(device), heatmap4.float().to(device)
with torch.no_grad():
    output4 = model(image4)
print(loss_fn(output4, heatmap4))

tensor(0.0770, device='cuda:0')


In [19]:
image4, heatmap4, landmarks4 = next(iter(train_loader))  
image4, heatmap4 = image4.float().to(device), heatmap4.float().to(device)
with torch.no_grad():
    output4 = model(image4)
print(loss_fn(output4, heatmap4))

tensor(0.0770, device='cuda:0')


In [20]:
image4, heatmap4, landmarks4 = next(iter(train_loader))  
image4, heatmap4 = image4.float().to(device), heatmap4.float().to(device)
with torch.no_grad():
    output4 = model(image4)
print(loss_fn(output4, heatmap4))

tensor(0.0770, device='cuda:0')


In [21]:
image4, heatmap4, landmarks4 = next(iter(train_loader))  
image4, heatmap4 = image4.float().to(device), heatmap4.float().to(device)
with torch.no_grad():
    output4 = model(image4)
print(loss_fn(output4, heatmap4))

tensor(0.0770, device='cuda:0')
